# A Metodologia


Como podemos observar no artigo, básicamente rodamos os métodos KNN e o SVM e guardamos os seus resultados.
### Imports necessários:

In [1]:
import os # Para manipular pastas
import numpy as np # Para manipular vetores e matrizes
from scipy.io import loadmat # Para carregar arquivos .mat
from sklearn.svm import SVC # Para treinar o SVM
from sklearn.neighbors import KNeighborsClassifier # Para treinar o KNN
from scipy.io import savemat # Para salvar arquivos .mat
import time # Para medir o tempo de execução
from joblib import Parallel, delayed
import pandas as pd # Para manipular dataframes
from skopt import BayesSearchCV # Para otimizar hiperparâmetros
from sklearn.model_selection import cross_val_score # Para validação cruzada

******
### Workflow (Com tunagem de parâmetros): 
1. [Importar os dados e definições básicas](#inicio) 
2. [Loop Principal](#loop)
   1. [A função metodo2](#metodo2)
   2. [A função metodo2 sem tunar](#metodo22)
3. [Salvando os resultados](#salvar)

******

```mermaid
flowchart LR
inicio[Importar os dados e definições básicas] --> loop{Vai tunar?}
loop -->|Sim| metodo2[Metodo2]
metodo2 --> salvar[Salvar os resultados]
loop -->|Não| metodo22[Metodo2 sem tunar]
metodo22 --> salvar
```






<div id='inicio'/>

##### Importando os dados e definições básicas

Começamos o nosso trabalho buscando os arquivos salvados em .mat e os importamos para python. Além disso, definimos os métodos e suas respectivas repetições que vão ser usadas no trabalho. 


In [2]:
# Pasta onde estão salvos as bases de dados
pasta = os.getcwd() + '/datatest'
nomedoarquivo = ''

# Métodos a serem rodados
metodos = ['KNN', 'SVC']
repeticoesPMetodo = [100, 100]

# Variaveis que vão ser usadas
qntMetodos = len(metodos)
if nomedoarquivo == '':
    arquivos = [f for f in os.listdir(pasta) if f.endswith('.mat')]
else:
    arquivos = [nomedoarquivo + '.mat']

tam = len(arquivos)
tabelas = np.empty((qntMetodos), dtype=object) # Tabela de resultados para cada método
tempos = np.empty((qntMetodos), dtype=object)

nomesBasesSaidas = ['']*tam


<div id='loop'>

##### Loop Principal

Aqui, para cada base de dados, eu rodo os classificadores e salvo os resultados na tabela. Eu chamo a função método2, que roda meus classificadores e me retorna a acurácia e o desvio padrão. 


$\lim_{x,y\rightarrow \} \frac{a^x}{x^p}$

In [4]:
for i in range(tam): # Itera cada uma das bases de dados
    print(i, ' - ', arquivos[i])
    nome = pasta + '/' + arquivos[i]
    nomesBasesSaidas[i] = arquivos[i]
    dataset = loadmat(nome) # Carrega a base de dados
    x = dataset['data_treino'] # Separa os dados de treino
    y = dataset['classe_treino']

    tabelaNomes = [arquivos[i][:-4]]*qntMetodos  # Nomes para cada tabela de resultados

    for j in range(qntMetodos):
        inicio = time.time() # Inicia a contagem do tempo
        mAcuracia, desvioPadrao = metodo2(x, y, metodos[j], repeticoesPMetodo[j])
        fim = time.time() # Finaliza a contagem do tempo
        tempo = fim - inicio
        tempoMedio = tempo / repeticoesPMetodo[j]
        tabela = pd.DataFrame([[mAcuracia, desvioPadrao, tempo ,tempoMedio]], 
                      columns=['Acuracia', 'Desvio Padrao', 'Tempo Total', 'Tempo Medio'], 
                      index=[tabelaNomes[j]]) # Cria a tabela de resultados
        if tabelas[j] is None: # Se for a primeira tabela, cria ela
            tabelas[j] = tabela
        else:
            tabelas[j] = tabelas[j].append(tabela)


0  -  bupa_dataset.mat


/home/gamaral/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/gamaral/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/gamaral/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/gamaral/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please cha

KeyboardInterrupt: 

<div id='metodo2'>

#### A função metodo2

O coração do nosso código, a função método2 recebe como parâmetros a base de dados, o classificador, o número de repetições. Ela retorna a acurácia e o desvio padrão.

Dentro dela hyperparametrizamos os nossos classificadores.

```python
def metodo2(X, Y, classifier_name, loop, cv=10):
    """_summary_

    Args:
        X (Array): Array de observações da base de dados.
        Y (Array (1D)): Vetor unidimensional de classes.
        classifier_name (List): Lista de classificadores (strings) a serem usados.
        loop (int): Quantidade de repetições.
        cv (int, optional): Numero de Folds da validação cruzada. Defaults to 10.

    Returns:
        mean_accuracy: Acurácia média.
        std_dev: Desvio padrão da acurácia.
    """
```

Em Python, para hyperparametrizar considerando todos os paramêtros, é necessário informar por meio de um dicionário quais parâmetros serão testados. Por exemplo, para o KNN, eu quero testar os valores de K de 1 a 10. Para isso, eu crio um dicionário com a chave sendo o nome do parâmetro e o valor sendo uma lista com os valores que eu quero testar. 

```python
classifiers = {
    'SVC': {
        'model': SVC(),
        'params': {
            'C': (1e-10, 1e10, 'log-uniform'),
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'degree': (1, 5),  # você pode querer ajustar o intervalo
            'gamma': (1e-10, 1e10, 'log-uniform'),
            'coef0': (0.0, 1.0)  # você pode querer ajustar o intervalo
        }
    }, 
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': (1, int(round(np.sqrt(len(X))))),
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'leaf_size': (1, 50),  # você pode querer ajustar o intervalo
            'p': [1, 2]
        }
    }
}
```

Dito isso, eu defino minhas variavéis como a acc e rodo o BayesSearchCV para Hyperparametrizar o método escolhido. Calculo a acurácia e salvo, repito esse processo a quantidade de vezes necessária e retorno a média e o desvio padrão.

```python
    acc = np.zeros(loop)
    model = classifiers[classifier_name]['model']
    search_space = classifiers[classifier_name]['params']
    for i in range(loop):
        opt = BayesSearchCV(model, search_space, n_iter=100, cv=cv, n_jobs=-1, random_state=i)
        opt.fit(X, Y)
        acc[i] = opt.best_score_
    mean_accuracy = np.mean(acc)
    std_dev = np.std(acc)
    return mean_accuracy, std_dev
```

In [3]:

def metodo2(X, Y, classifier_name, loop, cv=10):
    """_summary_

    Args:
        X (_type_): _description_
        Y (_type_): _description_
        classifier_name (_type_): _description_
        loop (_type_): _description_
        cv (int, optional): _description_. Defaults to 10.

    Returns:
        _type_: _description_
    """
    # Cria dicionário com os classificadores e seus espaços de busca de parâmetros
    np.int = np.int_
    classifiers = {
        'SVC': {
            'model': SVC(),
            'params': {
                'C': (1e-10, 1e10, 'log-uniform'),
                'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                'degree': (1, 5),  # você pode querer ajustar o intervalo
                'gamma': (1e-10, 1e10, 'log-uniform'),
                'coef0': (0.0, 1.0)  # você pode querer ajustar o intervalo
            }
        }, 
        'KNN': {
            'model': KNeighborsClassifier(),
            'params': {
                'n_neighbors': (1, int(round(np.sqrt(len(X))))),
                'weights': ['uniform', 'distance'],
                'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                'leaf_size': (1, 50),  # você pode querer ajustar o intervalo
                'p': [1, 2]
            }
        }
    }
    acc = np.zeros(loop)
    model = classifiers[classifier_name]['model']
    search_space = classifiers[classifier_name]['params']
    for i in range(loop):
        opt = BayesSearchCV(model, search_space, n_iter=100, cv=cv, n_jobs=-1, random_state=i)
        opt.fit(X, Y)
        acc[i] = opt.best_score_
    mean_accuracy = np.mean(acc)
    std_dev = np.std(acc)
    return mean_accuracy, std_dev


<div id='metodo22'>

#### A função metodo2 sem tunar
O mesmo que a função anterior, porém sem hyperparametrizar.

```python
def metodo2(X, Y, classifier_name, loop, cv=10):
    """_summary_

    Args:
        X (Array): Array de observações da base de dados.
        Y (Array (1D)): Vetor unidimensional de classes.
        classifier_name (List): Lista de classificadores (strings) a serem usados.
        loop (int): Quantidade de repetições.
        cv (int, optional): Numero de Folds da validação cruzada. Defaults to 10.

    Returns:
        mean_accuracy: Acurácia média.
        std_dev: Desvio padrão da acurácia.
    """
```



In [6]:
def metodo2(X, Y, classifier_name, loop, cv=10):
    """_summary_

    Args:
        X (_type_): _description_
        Y (_type_): _description_
        classifier_name (_type_): _description_
        loop (_type_): _description_
        cv (int, optional): _description_. Defaults to 10.

    Returns:
        _type_: _description_
    """
    # Cria dicionário com os classificadores e seus parametros
    np.int = np.int_
    classifiers = {
        'SVC': {
            'model': SVC(),
        },
        'KNN': {
            'model': KNeighborsClassifier(),
        },
    }
    acc = np.zeros(loop)
    model = classifiers[classifier_name]['model']
    for i in range(loop):
        mdl = model.fit(X, Y)
        acc[i] = cross_val_score(mdl, X, Y, cv=cv).mean()
    mean_accuracy = np.mean(acc)
    std_dev = np.std(acc)
    return mean_accuracy, std_dev

<div id='salvar'>

##### Salvando os resultados

Com tudo calculado e armazenado, basta criar os arquivos de saida e salvar os resultados. 

In [ ]:
nomes_metodos = "".join(metodos)
pastaResultados = os.getcwd() + '/ResultadosPython2'

# Verifica se a pasta de resultados existe, se não, cria ela
if not os.path.isdir(pastaResultados):
    os.makedirs(pastaResultados)

for i in range(qntMetodos):
    tabela = tabelas[i]
    valType = 'KFold'
    valParam = '10'
    # Salva a tabela de resultados em .mat usando scipy
    filename = pastaResultados + '/' + metodos[i] + str(nomesBasesSaidas)  + '_' + valType + '_' + str(valParam) + '.mat'
    mdict = {'tabela': tabela}
    savemat(filename, mdict)